In [ ]:
### Python Notebook to Test Openquake and Pygmm
# Daniel Trugman, 2025


In [ ]:
### Imports

# base
import pandas as pd
import numpy as np
import os
import math
from obspy.clients.fdsn import Client
import rasterio
from scipy.spatial import cKDTree
from datetime import datetime

# plotting
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
#sns.set_style("darkgrid")

# openquake - generic
# - https://docs.openquake.org/oq-engine/3.5/baselib.html#installation
#  - worked ok as !pip install openquake.engine in python 3.10 environment...
#    - but needed to add fiona afterward with mamba
# - for documentation see this:
# - https://docs.openquake.org/oq-engine/3.5/openquake.hazardlib.html
from openquake.hazardlib.contexts import RuptureContext
from openquake.hazardlib.contexts import DistancesContext
from openquake.hazardlib.contexts import SitesContext
from openquake.hazardlib import imt, const

# active crustal models
from openquake.hazardlib.gsim.abrahamson_2014 import AbrahamsonEtAl2014
from openquake.hazardlib.gsim.boore_2014 import BooreEtAl2014
from openquake.hazardlib.gsim.campbell_bozorgnia_2014 import CampbellBozorgnia2014
from openquake.hazardlib.gsim.chiou_youngs_2014 import ChiouYoungs2014


In [ ]:
### Setup for Calculations


## Define Rupture Context
rctx = RuptureContext()
rctx.mag = 5.7
rctx.rake = -19
rctx.dip = 80
rctx.ztor = 0.1 #guessing
rctx.width = 10 # guessing
rctx.hypo_depth = 9.3 

## Define Distance Context (assume fault-perpendicular sites)
dctx = DistancesContext()
rjbs = np.linspace(0.0, 320.0, 321) # rjb values in km
rrups = np.sqrt(rjbs**2 + rctx.ztor**2) # calculated rupture distance for a vertical fault
npts = rrups.size # number of distance points
dctx.rrup = rrups # Closest distance to rupture surface
dctx.rjb = rjbs # Distance to rupture’s surface projection
dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                         #  (essentially, how far a location is "along the fault" from the rupture termination point)


## Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
#    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                'vs30measured': np.zeros(npts, dtype=bool), # true or false, not sure how this is used
                'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                }
sitecollection = pd.DataFrame(sitecol_dict)
sctx = SitesContext(sitecol=sitecollection)   

In [ ]:
### Calculate Ground Motions

# define metrics
#IMT = imt.PGA()
IMT = imt.PGV()
#IMT = imt.SA(period=0.1)
uncertaintytype = const.StdDev.TOTAL

# instantiate GMMs
ASK14 = AbrahamsonEtAl2014()
BSSA14 = BooreEtAl2014()
CB14 = CampbellBozorgnia2014()
CY14 = ChiouYoungs2014()

# calculate ln ground motions for each
ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])

# average across all four for active crustal values
ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1


In [ ]:
### Find Observed Data

df = pd.read_csv('nn00888580_default_metrics_rotd(percentile=50.0).csv')
OBSdrup = df['RuptureDistance']
OBSpga = df['PGA']
OBSpgv = df['PGV']

In [ ]:
### Plot Model Results

# figure setup
fig, axi = plt.subplots(figsize=(6,4))
axi.set_facecolor("gainsboro")

# plot GMPE
ly1 = ln_median_ac14 - ln_sd_ac14
ly2 = ln_median_ac14 + ln_sd_ac14
axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
plt.scatter(OBSdrup, OBSpgv, s=5, label="Parker Butte M5.7 Observed Motions", zorder=0) #OBSpgv or OBSpga
axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=2)
#axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
#axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)

# formatting
axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
axi.set_ylabel("PGV [cm/s]",fontsize=16)
axi.grid(lw=0.3)
axi.legend(loc="upper right",fontsize=11)
axi.set_xlim(9,320)

# Save the figure with a specified DPI
plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show results
plt.show()


### To Do

- basin term and vs30 defaults
- optimize rupture context
- optimize distance metrics?




In [ ]:
#All together (RESIDUAL CALCULATIONS PGV)
 
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []

#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():
    event = row['eventID']
    magnitude = row['EarthquakeMagnitude']
    depth = row['EarthquakeDepth']
    
    # download more rupture data for the event
    cat = client.get_events(eventid=f"{event}")
    cat = cat[0]
    
    #find event specific data from gmprocess (Reno1)
    base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
    filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
    file_path = os.path.join(base_dir, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df = df.sort_values(by='RuptureDistance', ascending =True)
        df = df.reset_index(drop=True)
        
    ### Define Rupture Context
    
    rctx = RuptureContext()
    rctx.mag = magnitude

    #try to find focal mechanism to add rake and dip
    try:
        mech = cat.focal_mechanisms[0]
        nop = mech.nodal_planes['nodal_plane_1']
        rctx.rake = nop['rake']
        rctx.dip = nop['dip']
        print(f'Found a focal mechanism for {magnitude} {event}')
    except:
        rctx.rake = 0
        rctx.dip = 45
        
    # assume square ruptures with identical stress drop for every event
    '''WHAT DOES SQUARE RUPTURE BIAS TOWARD IN TERMS OF GMM COMPARED TO RANGE OF RUPTURE STYLES? GUESSING IT LEADS TO SMALLER PREDICTED MOTION
        #THAN REALISTIC RUPTURE SCENARIOS DUE TO GREATER DEPTH AND LESS WIDTH THAN RECTANGULAR PLANES
    IT WOULD BE INTERESTING TO TRY THIS WITH CIRCLES/ELLIPSES TOO AND COMPARE THE RESULTING RESIDUALS'''
    moment = 10 ** (1.5*(magnitude+10.7))
    stressdrop = 3*10**6 #Pa
    L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
    L = L/1000 #km 
    rctx.ztor = depth-L/2 #from hypo to edge of square
    rctx.width =L #length of square
    rctx.hypo_depth = depth
    
    ### Define Distance Context (assume fault-perpendicular sites)
    
    dctx = DistancesContext()
    rjbs = df['JoynerBooreDistance'] # rjb values in km
    rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
    npts = rrups.size # number of distance points
    dctx.rrup = rrups # Closest distance to rupture surface
    dctx.rjb = rjbs # Distance to rupture’s surface projection
    dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
    dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                             #  (essentially, how far a location is "along the fault" from the rupture termination point)
    
    ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
    #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
    #Find Vs30 measurements from USGS raster
    sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                    'vs30':760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                    'vs30measured': np.zeros(npts, dtype=bool), # true or false, not sure how this is used
                    'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                    'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                    }
    sitecollection = pd.DataFrame(sitecol_dict)
    sctx = SitesContext(sitecol=sitecollection)   
    
    ### Calculate Ground Motions
    
    # define metrics
    #IMT = imt.PGA() #IS THIS IN G OR %G
    IMT = imt.PGV()
    #IMT = imt.SA(period=0.1)
    uncertaintytype = const.StdDev.TOTAL
    
    # instantiate GMMs
    ASK14 = AbrahamsonEtAl2014()
    BSSA14 = BooreEtAl2014()
    CB14 = CampbellBozorgnia2014()
    CY14 = ChiouYoungs2014()
    
    # calculate ln ground motions for each
    ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    
    # average across all four for active crustal values
    ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
    ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1    

    ### Find Observed Data
    
    # gather observed ground motion metrics
    OBSdrup = df['RuptureDistance']
    OBSpga = df['PGA']*0.01 #%g
    OBSpgv = df['PGV'] #cm/s
    OBSsa = df['SA(T=1.0000, D=0.050)']

    ''' identify which metric is being worked with '''
    ObservedMetric = OBSpgv
    
    # calculate residuals with estimated data
    residuals = []
    for i in range(len(ln_median_ac14)):
        residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14 #normalized log residual
        residuals.append(residual)
    residuals = np.array(residuals)
    mean_residual = np.mean(residuals)
    totalresiduals.append(mean_residual)
    mean_residual = round(mean_residual, 4)

    ### Plot Model Results
    
    # figure setup
    fig, axi = plt.subplots(figsize=(6,4))
    axi.set_facecolor("gainsboro")
    
    # plot GMPE
    ly1 = ln_median_ac14 - ln_sd_ac14
    ly2 = ln_median_ac14 + ln_sd_ac14
    axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
    plt.scatter(OBSdrup, ObservedMetric, s=5, label=f"{event} Observed Motions", zorder=0)
    axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=2)
    #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
    #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
    
    # formatting
    plt.title(f'M{magnitude} {event}: residual={mean_residual}')
    axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
    if ObservedMetric.equals(OBSpgv):
        axi.set_ylabel("PGV [cm/s]",fontsize=16)
    if ObservedMetric.equals(OBSpga):
        axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
    if ObservedMetric.equals(OBSsa):
        axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
    axi.grid(lw=0.3)
    axi.legend(loc="upper right",fontsize=11)
    axi.set_xlim(9,320)

# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')


In [ ]:
print(len(totalresiduals))

In [ ]:
#All together (RESIDUAL CALCULATIONS PGA)
 
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []

#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():
    event = row['eventID']
    magnitude = row['EarthquakeMagnitude']
    depth = row['EarthquakeDepth']
    
    # download more rupture data for the event
    cat = client.get_events(eventid=f"{event}")
    cat = cat[0]
    
    #find event specific data from gmprocess (Reno1)
    base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
    filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
    file_path = os.path.join(base_dir, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df = df.sort_values(by='RuptureDistance', ascending =True)
        df = df.reset_index(drop=True)
        
    ### Define Rupture Context
    
    rctx = RuptureContext()
    rctx.mag = magnitude

    #try to find focal mechanism to add rake and dip
    try:
        mech = cat.focal_mechanisms[0]
        nop = mech.nodal_planes['nodal_plane_1']
        rctx.rake = nop['rake']
        rctx.dip = nop['dip']
        print(f'Found a focal mechanism for {magnitude} {event}')
    except:
        rctx.rake = 0
        rctx.dip = 90
        
    # assume square ruptures with identical stress drop for every event
    '''WHAT DOES SQUARE RUPTURE BIAS TOWARD IN TERMS OF GMM COMPARED TO RANGE OF RUPTURE STYLES? GUESSING IT LEADS TO SMALLER PREDICTED MOTION
        #THAN REALISTIC RUPTURE SCENARIOS DUE TO GREATER DEPTH AND LESS WIDTH THAN RECTANGULAR PLANES
    IT WOULD BE INTERESTING TO TRY THIS WITH CIRCLES/ELLIPSES TOO AND COMPARE THE RESULTING RESIDUALS'''
    moment = 10 ** (1.5*(magnitude+10.7))
    stressdrop = 3*10**6 #Pa
    L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
    L = L/1000 #km 
    rctx.ztor = depth-L/2 #from hypo to edge of square
    rctx.width =L #length of square
    rctx.hypo_depth = depth
    
    ### Define Distance Context (assume fault-perpendicular sites)
    
    dctx = DistancesContext()
    rjbs = df['JoynerBooreDistance'] # rjb values in km
    rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
    npts = rrups.size # number of distance points
    dctx.rrup = rrups # Closest distance to rupture surface
    dctx.rjb = rjbs # Distance to rupture’s surface projection
    dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
    dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                             #  (essentially, how far a location is "along the fault" from the rupture termination point)
    
    ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
    #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
    sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                    'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                    'vs30measured': np.zeros(npts, dtype=bool), # true or false, not sure how this is used
                    'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                    'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                    }
    sitecollection = pd.DataFrame(sitecol_dict)
    sctx = SitesContext(sitecol=sitecollection)   
    
    ### Calculate Ground Motions
    
    # define metrics
    #IMT = imt.PGA() #IS THIS IN G OR %G
    #IMT = imt.PGV()
    IMT = imt.SA(period=0.1)
    uncertaintytype = const.StdDev.TOTAL
    
    # instantiate GMMs
    ASK14 = AbrahamsonEtAl2014()
    BSSA14 = BooreEtAl2014()
    CB14 = CampbellBozorgnia2014()
    CY14 = ChiouYoungs2014()
    
    # calculate ln ground motions for each
    ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    
    # average across all four for active crustal values
    ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
    ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1    

    ### Find Observed Data
    
    # gather observed ground motion metrics
    OBSdrup = df['RuptureDistance']
    OBSpga = df['PGA']*0.01 #%g
    OBSpgv = df['PGV'] #cm/s
    OBSsa = df['SA(T=1.0000, D=0.050)']

    ''' identify which metric is being worked with '''
    ObservedMetric = OBSsa
    
    # calculate residuals with estimated data
    residuals = []
    for i in range(len(ln_median_ac14)):
        residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14 #normalized log residual
        residuals.append(residual)
    residuals = np.array(residuals)
    mean_residual = np.mean(residuals)
    totalresiduals.append(mean_residual)
    mean_residual = round(mean_residual, 4)

    ### Plot Model Results
    
    # figure setup
    fig, axi = plt.subplots(figsize=(6,4))
    axi.set_facecolor("gainsboro")
    
    # plot GMPE
    ly1 = ln_median_ac14 - ln_sd_ac14
    ly2 = ln_median_ac14 + ln_sd_ac14
    axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
    plt.scatter(OBSdrup, ObservedMetric, s=5, label=f"{event} Observed Motions", zorder=0)
    axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=2)
    #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
    #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
    
    # formatting
    plt.title(f'M{magnitude} {event}: residual={mean_residual}')
    axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
    if ObservedMetric.equals(OBSpgv):
        axi.set_ylabel("PGV [cm/s]",fontsize=16)
    if ObservedMetric.equals(OBSpga):
        axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
    if ObservedMetric.equals(OBSsa):
        axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
    axi.grid(lw=0.3)
    axi.legend(loc="upper right",fontsize=11)
    axi.set_xlim(9,320)

# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')


In [ ]:
#All together (RESIDUAL CALCULATIONS PGA)
 
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []

#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():
    event = row['eventID']
    magnitude = row['EarthquakeMagnitude']
    depth = row['EarthquakeDepth']
    
    # download more rupture data for the event
    cat = client.get_events(eventid=f"{event}")
    cat = cat[0]
    
    #find event specific data from gmprocess (Reno1)
    base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
    filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
    file_path = os.path.join(base_dir, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df = df.sort_values(by='RuptureDistance', ascending =True)
        df = df.reset_index(drop=True)
        
    ### Define Rupture Context
    
    rctx = RuptureContext()
    rctx.mag = magnitude

    #try to find focal mechanism to add rake and dip
    try:
        mech = cat.focal_mechanisms[0]
        nop = mech.nodal_planes['nodal_plane_1']
        rctx.rake = nop['rake']
        rctx.dip = nop['dip']
        print(f'Found a focal mechanism for {magnitude} {event}')
    except:
        rctx.rake = 0
        rctx.dip = 90
        
    # assume square ruptures with identical stress drop for every event
    '''WHAT DOES SQUARE RUPTURE BIAS TOWARD IN TERMS OF GMM COMPARED TO RANGE OF RUPTURE STYLES? GUESSING IT LEADS TO SMALLER PREDICTED MOTION
        #THAN REALISTIC RUPTURE SCENARIOS DUE TO GREATER DEPTH AND LESS WIDTH THAN RECTANGULAR PLANES
    IT WOULD BE INTERESTING TO TRY THIS WITH CIRCLES/ELLIPSES TOO AND COMPARE THE RESULTING RESIDUALS'''
    moment = 10 ** (1.5*(magnitude+10.7))
    stressdrop = 3*10**6 #Pa
    L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
    L = L/1000 #km 
    rctx.ztor = depth-L/2 #from hypo to edge of square
    rctx.width =L #length of square
    rctx.hypo_depth = depth
    
    ### Define Distance Context (assume fault-perpendicular sites)
    
    dctx = DistancesContext()
    rjbs = df['JoynerBooreDistance'] # rjb values in km
    rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
    npts = rrups.size # number of distance points
    dctx.rrup = rrups # Closest distance to rupture surface
    dctx.rjb = rjbs # Distance to rupture’s surface projection
    dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
    dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                             #  (essentially, how far a location is "along the fault" from the rupture termination point)
    
    ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
    #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
    sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                    'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                    'vs30measured': np.zeros(npts, dtype=bool), # true or false, not sure how this is used
                    'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                    'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                    }
    sitecollection = pd.DataFrame(sitecol_dict)
    sctx = SitesContext(sitecol=sitecollection)   
    
    ### Calculate Ground Motions
    
    # define metrics
    IMT = imt.PGA() #IS THIS IN G OR %G
    #IMT = imt.PGV()
    #IMT = imt.SA(period=0.1)
    uncertaintytype = const.StdDev.TOTAL
    
    # instantiate GMMs
    ASK14 = AbrahamsonEtAl2014()
    BSSA14 = BooreEtAl2014()
    CB14 = CampbellBozorgnia2014()
    CY14 = ChiouYoungs2014()
    
    # calculate ln ground motions for each
    ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    
    # average across all four for active crustal values
    ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
    ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1    

    ### Find Observed Data
    
    # gather observed ground motion metrics
    OBSdrup = df['RuptureDistance']
    OBSpga = df['PGA']*0.01 #%g
    OBSpgv = df['PGV'] #cm/s
    OBSsa = df['SA(T=1.0000, D=0.050)']

    ''' identify which metric is being worked with '''
    ObservedMetric = OBSpga
    
    # calculate residuals with estimated data
    residuals = []
    for i in range(len(ln_median_ac14)):
        residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14 #normalized log residual
        residuals.append(residual)
    residuals = np.array(residuals)
    mean_residual = np.mean(residuals)
    totalresiduals.append(mean_residual)
    mean_residual = round(mean_residual, 4)

    ### Plot Model Results
    
    # figure setup
    fig, axi = plt.subplots(figsize=(6,4))
    axi.set_facecolor("gainsboro")
    
    # plot GMPE
    ly1 = ln_median_ac14 - ln_sd_ac14
    ly2 = ln_median_ac14 + ln_sd_ac14
    axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
    plt.scatter(OBSdrup, ObservedMetric, s=5, label=f"{event} Observed Motions", zorder=0)
    axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=2)
    #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
    #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
    
    # formatting
    plt.title(f'M{magnitude} {event}: residual={mean_residual}')
    axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
    if ObservedMetric.equals(OBSpgv):
        axi.set_ylabel("PGV [cm/s]",fontsize=16)
    if ObservedMetric.equals(OBSpga):
        axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
    if ObservedMetric.equals(OBSsa):
        axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
    axi.grid(lw=0.3)
    axi.legend(loc="upper right",fontsize=11)
    axi.set_xlim(9,320)

# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')

In [ ]:
#All together (PLOTTING)
 
### Setup for Calculations

eventdata = pd.read_csv("ListofCompletedEvents.csv")
for _, row in eventdata.iterrows():
    event = row['eventID']
    magnitude = row['EarthquakeMagnitude']
    depth = row['EarthquakeDepth'] 

    ## Define Rupture Context
    rctx = RuptureContext()
    rctx.mag = magnitude #PB 5.7
    rctx.rake = 0 #PB -19
    rctx.dip = 90 #PB 80
    moment = 10 ** (1.5*(magnitude+10.7))
    stressdrop = 3*10**6 #Pa
    L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
    L = L/1000 #km
    
    rctx.ztor = depth-L/2 #guessing
    rctx.width =L # guessing
    rctx.hypo_depth = depth #PB 9.3 
    
    ## Define Distance Context (assume fault-perpendicular sites)
    dctx = DistancesContext()
    rjbs = np.linspace(0.0, 320.0, 321) # rjb values in km
    rrups = np.sqrt(rjbs**2 + rctx.ztor**2) # calculated rupture distance for a vertical fault
    npts = rrups.size # number of distance points
    dctx.rrup = rrups # Closest distance to rupture surface
    dctx.rjb = rjbs # Distance to rupture’s surface projection
    dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
    dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                             #  (essentially, how far a location is "along the fault" from the rupture termination point)
    
    
    ## Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
    #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
    sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                    'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                    'vs30measured': np.zeros(npts, dtype=bool), # true or false, not sure how this is used
                    'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                    'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                    }
    sitecollection = pd.DataFrame(sitecol_dict)
    sctx = SitesContext(sitecol=sitecollection)   
    
    ### Calculate Ground Motions
    
    # define metrics
    #IMT = imt.PGA()
    IMT = imt.PGV()
    #IMT = imt.SA(period=0.1)
    uncertaintytype = const.StdDev.TOTAL
    
    # instantiate GMMs
    ASK14 = AbrahamsonEtAl2014()
    BSSA14 = BooreEtAl2014()
    CB14 = CampbellBozorgnia2014()
    CY14 = ChiouYoungs2014()
    
    # calculate ln ground motions for each
    ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    
    # average across all four for active crustal values
    ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
    ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1
    
    ### Find Observed Data
    base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"     #FIX THIS WITH EVENT DIRECTORY STUFF FROM ANALYZER
    filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
    file_path = os.path.join(base_dir, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

    # gather observed ground motion metrics
    OBSdrup = df['RuptureDistance']
    OBSpga = df['PGA'] #%g
    OBSpgv = df['PGV'] #cm/s
    
    ### Plot Model Results
    
    # figure setup
    fig, axi = plt.subplots(figsize=(6,4))
    axi.set_facecolor("gainsboro")
    
    # plot GMPE
    ly1 = ln_median_ac14 - ln_sd_ac14
    ly2 = ln_median_ac14 + ln_sd_ac14
    axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
    plt.scatter(OBSdrup, OBSpgv, s=5, label=f"M{magnitude} {event} Observed Motions", zorder=0) #OBSpgv or OBSpga
    axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=2)
    #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
    #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
    
    # formatting
    axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
    axi.set_ylabel("PGV [cm/s]",fontsize=16)
    #axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x
    axi.grid(lw=0.3)
    axi.legend(loc="upper right",fontsize=11)
    axi.set_xlim(9,320)

# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show results
plt.show()


In [ ]:
### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
    #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
    #Find Vs30 measurements from USGS raster
    sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                    'vs30':760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                    'vs30measured': np.zeros(npts, dtype=bool), # true or false, not sure how this is used
                    'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                    'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                    }
    sitecollection = pd.DataFrame(sitecol_dict)
    sctx = SitesContext(sitecol=sitecollection)

In [ ]:
print(df)

In [ ]:
#Prepare files for Vs30 analysis
# Load Vs30 CSV
vs30_df = pd.read_csv(f'/Users/carterdills/Documents/QGIS/RenoVS30.csv')
# Threshold distance in degrees
threshold_deg = 0.0001
# Build KDTree from vs30 points
tree_coords = vs30_df[['Latitude', 'Longitude']].to_numpy()
tree = cKDTree(tree_coords)
# Load raster
raster = rasterio.open(f'/Users/carterdills/Documents/vs30/vs30_mosaic.tif')
raster_array = raster.read(1)
transform = raster.transform

# Prepare results
vs30_values = []
measured_flags = []

# Iterate over stations
for _, station in df.iterrows():
    lat = station['StationLatitude']
    lon = station['StationLongitude']
    
    # Query tree for neighbors within the threshold
    distances, indices = tree.query([[lat, lon]], k=1, distance_upper_bound=threshold_deg)
    index = indices[0]
    
    if index != tree.n:  # Found a valid neighbor
        vs30 = vs30_df.iloc[index]['Vs30']
        measured = True
    else:
        # Fallback to raster
        row, col = raster.index(lon, lat)
        if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
            vs30 = raster_array[row, col]
        else:
            vs30 = np.nan
        measured = False
    
    vs30_values.append(vs30)
    measured_flags.append(measured)

# Add columns to stations DataFrame
df['Vs30'] = vs30_values
df['measured'] = measured_flags

In [ ]:
#All together (RESIDUAL CALCULATIONS PGV, Vs30 colorbar, can use time subset)
 
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []

#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")
#eventdata = pd.read_csv("EventsPracticeSubset.csv")

#Prepare files for Vs30 analysis
# Load Vs30 CSV
vs30_df = pd.read_csv(f'/Users/carterdills/Documents/QGIS/RenoVS30.csv')
# Threshold distance in degrees
threshold_deg = 0.0001
# Build KDTree from vs30 points
tree_coords = vs30_df[['Latitude', 'Longitude']].to_numpy()
tree = cKDTree(tree_coords)
# Load raster
raster = rasterio.open(f'/Users/carterdills/Documents/vs30/vs30_mosaic.tif')
raster_array = raster.read(1)
transform = raster.transform

pdf_pages = PdfPages('PGVoutputs.pdf')

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():

    #Comment this section out if you just want to use whole dataset
    dt = datetime.fromisoformat(row['EarthquakeTime'])
    start = datetime.fromisoformat('2008-01-01T00:00:00.000000Z')
    end = datetime.fromisoformat('2009-01-01T00:00:00.000000Z')
    # Check if dt is between start and end
    if start <= dt <= end:
        
        event = row['eventID']
        magnitude = row['EarthquakeMagnitude']
        depth = row['EarthquakeDepth']
        
        # download more rupture data for the event
        cat = client.get_events(eventid=f"{event}")
        cat = cat[0]
        
        #find event specific data from gmprocess (Reno1)
        base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
        filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
        file_path = os.path.join(base_dir, filename)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df = df.sort_values(by='RuptureDistance', ascending =True)
            df = df.reset_index(drop=True)
            
        ### Define Rupture Context
        
        rctx = RuptureContext()
        rctx.mag = magnitude
    
        #try to find focal mechanism to add rake and dip
        try:
            mech = cat.focal_mechanisms[0]
            nop = mech.nodal_planes['nodal_plane_1']
            rctx.rake = nop['rake']
            rctx.dip = nop['dip']
            print(f'Found a focal mechanism for {magnitude} {event}')
        except:
            rctx.rake = 0
            rctx.dip = 45
            
        # assume square ruptures with identical stress drop for every event
        '''WHAT DOES SQUARE RUPTURE BIAS TOWARD IN TERMS OF GMM COMPARED TO RANGE OF RUPTURE STYLES? GUESSING IT LEADS TO SMALLER PREDICTED MOTION
            #THAN REALISTIC RUPTURE SCENARIOS DUE TO GREATER DEPTH AND LESS WIDTH THAN RECTANGULAR PLANES
        IT WOULD BE INTERESTING TO TRY THIS WITH CIRCLES/ELLIPSES TOO AND COMPARE THE RESULTING RESIDUALS'''
        moment = 10 ** (1.5*(magnitude+10.7))
        stressdrop = 3*10**6 #Pa
        L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
        L = L/1000 #km 
        rctx.ztor = depth-L/2 #from hypo to edge of square
        rctx.width =L #length of square
        rctx.hypo_depth = depth
        
        ### Define Distance Context (assume fault-perpendicular sites)
        
        dctx = DistancesContext()
        rjbs = df['JoynerBooreDistance'] # rjb values in km
        rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
        npts = rrups.size # number of distance points
        dctx.rrup = rrups # Closest distance to rupture surface
        dctx.rjb = rjbs # Distance to rupture’s surface projection
        dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
        dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                                 #  (essentially, how far a location is "along the fault" from the rupture termination point)
        
        ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
        
        #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
        # Prepare results
        vs30_values = []
        measured_flags = []
    
        #Find Vs30 measurements from USGS raster
        # Iterate over stations
        for _, station in df.iterrows():
            lat = station['StationLatitude']
            lon = station['StationLongitude']
            
            # Query tree for neighbors within the threshold
            distances, indices = tree.query([[lat, lon]], k=1, distance_upper_bound=threshold_deg)
            index = indices[0]
            
            if index != tree.n:  # Found a valid neighbor
                vs30 = vs30_df.iloc[index]['Vs30']
                measured = True
            else:
                # Fallback to raster
                row, col = raster.index(lon, lat)
                if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
                    vs30 = raster_array[row, col]
                else:
                    vs30 = np.nan
                measured = False
            
            vs30_values.append(vs30)
            measured_flags.append(measured)
        
        # Add columns to stations DataFrame
        df['Vs30'] = vs30_values
        df['measured'] = measured_flags
        sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                        'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                        'vs30measured': df['measured'], # true or false, not sure how this is used
                        'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                        'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                        }
        sitecollection = pd.DataFrame(sitecol_dict)
        sctx = SitesContext(sitecol=sitecollection)   
        
        ### Calculate Ground Motions
        
        # define metrics
        #IMT = imt.PGA() #IS THIS IN G OR %G
        IMT = imt.PGV()
        #IMT = imt.SA(period=0.1)
        uncertaintytype = const.StdDev.TOTAL
        
        # instantiate GMMs
        ASK14 = AbrahamsonEtAl2014()
        BSSA14 = BooreEtAl2014()
        CB14 = CampbellBozorgnia2014()
        CY14 = ChiouYoungs2014()
        
        # calculate ln ground motions for each
        ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        
        # average across all four for active crustal values
        ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
        ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1    
    
        ### Find Observed Data
        
        # gather observed ground motion metrics
        OBSdrup = df['RuptureDistance']
        OBSpga = df['PGA']*0.01 #%g
        OBSpgv = df['PGV'] #cm/s
        OBSsa = df['SA(T=1.0000, D=0.050)']
    
        ''' identify which metric is being worked with '''
        ObservedMetric = OBSpgv
        
        # calculate residuals with estimated data
        residuals = []
        for i in range(len(ln_median_ac14)):
            residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14[i] #normalized log residual
            residuals.append(residual)
        residuals = np.array(residuals)
        df['Residuals']= residuals
        mean_residual = np.mean(residuals)
        totalresiduals.append(mean_residual)
        mean_residual = round(mean_residual, 4)
    
        ### Plot Model Results
        
        # figure setup
        fig, axi = plt.subplots(figsize=(6,4))
        axi.set_facecolor("gainsboro")
         # Create a colormap
        cmap = plt.colormaps['seismic']
        # Normalize the values to the range 0-1 for the colormap
        norm = plt.Normalize(vmin=0, vmax=1520)
        
        # plot GMPE
        ly1 = ln_median_ac14 - ln_sd_ac14
        ly2 = ln_median_ac14 + ln_sd_ac14
        axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
        plt.scatter(OBSdrup, ObservedMetric, c=vs30_values, cmap=cmap, norm=norm, s=5, label=f"{event} Observed Motions", zorder=2)
        #plt.scatter(rrups,np.exp(ln_median_ac14),c=vs30_values, cmap='viridis', s=3)
        axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=0)
        #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
        #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
        
        # formatting
        plt.title(f'M{magnitude} {event}: residual={mean_residual}')
        axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
        if ObservedMetric.equals(OBSpgv):
            axi.set_ylabel("PGV [cm/s]",fontsize=16)
        if ObservedMetric.equals(OBSpga):
            axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
        if ObservedMetric.equals(OBSsa):
            axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
        axi.grid(lw=0.3)
        axi.legend(loc="upper right",fontsize=11)
        axi.set_xlim(9,320)
        plt.xscale('log')
        plt.yscale('log')
        plt.colorbar(label='Vs30')

        pdf_pages.savefig(fig)
        plt.close(fig)
    else:
        print('Out of Date Range')
# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
#plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
#plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')
pdf_pages.close()

In [ ]:
#All together (RESIDUAL CALCULATIONS PGV, SNR colorbar, can use time or mag subset)
 
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []

#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")
#eventdata = pd.read_csv("EventsPracticeSubset.csv")

#Prepare files for Vs30 analysis
# Load Vs30 CSV
vs30_df = pd.read_csv(f'/Users/carterdills/Documents/QGIS/RenoVS30.csv')
# Threshold distance in degrees
threshold_deg = 0.0001
# Build KDTree from vs30 points
tree_coords = vs30_df[['Latitude', 'Longitude']].to_numpy()
tree = cKDTree(tree_coords)
# Load raster
raster = rasterio.open(f'/Users/carterdills/Documents/vs30/vs30_mosaic.tif')
raster_array = raster.read(1)
transform = raster.transform

pdf_pages = PdfPages('PGVoutputs.pdf')

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():

    '''#Comment this section out if you just want to use whole event dataset
    dt = datetime.fromisoformat(row['EarthquakeTime'])
    start = datetime.fromisoformat('2008-01-01T00:00:00.000000Z')
    end = datetime.fromisoformat('2009-01-01T00:00:00.000000Z')
    # Check if dt is between start and end
    if start <= dt <= end:'''

    mag = row['EarthquakeMagnitude']
    minmag = 3
    maxmag = 5.7
    # Check if mag is between min and max
    if minmag <= mag <= maxmag:
        
        event = row['eventID']
        magnitude = row['EarthquakeMagnitude']
        depth = row['EarthquakeDepth']
        
        # download more rupture data for the event
        cat = client.get_events(eventid=f"{event}")
        cat = cat[0]
        
        #find event specific data from gmprocess (Reno1)
        base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
        filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
        file_path = os.path.join(base_dir, filename)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df = df.sort_values(by='RuptureDistance', ascending =True)
            df = df.reset_index(drop=True)
            
        ### Define Rupture Context
        
        rctx = RuptureContext()
        rctx.mag = magnitude
    
        #try to find focal mechanism to add rake and dip
        try:
            mech = cat.focal_mechanisms[0]
            nop = mech.nodal_planes['nodal_plane_1']
            rctx.rake = nop['rake']
            rctx.dip = nop['dip']
            print(f'Found a focal mechanism for {magnitude} {event}')
        except:
            rctx.rake = 0
            rctx.dip = 45
            
        # assume square ruptures with identical stress drop for every event
        moment = 10 ** (1.5*(magnitude+10.7))
        stressdrop = 3*10**6 #Pa
        L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
        L = L/1000 #km 
        rctx.ztor = depth-L/2 #from hypo to edge of square
        rctx.width =L #length of square
        rctx.hypo_depth = depth
        
        ### Define Distance Context (assume fault-perpendicular sites)
        
        dctx = DistancesContext()
        rjbs = df['JoynerBooreDistance'] # rjb values in km
        rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
        npts = rrups.size # number of distance points
        dctx.rrup = rrups # Closest distance to rupture surface
        dctx.rjb = rjbs # Distance to rupture’s surface projection
        dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
        dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                                 #  (essentially, how far a location is "along the fault" from the rupture termination point)
        
        ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
        
        #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
        # Prepare results
        vs30_values = []
        measured_flags = []
    
        #Find Vs30 measurements from USGS raster
        # Iterate over stations
        for _, station in df.iterrows():
            lat = station['StationLatitude']
            lon = station['StationLongitude']
            # Query tree for neighbors within the threshold
            distances, indices = tree.query([[lat, lon]], k=1, distance_upper_bound=threshold_deg)
            index = indices[0]
            if index != tree.n:  # Found a valid neighbor
                vs30 = vs30_df.iloc[index]['Vs30']
                measured = True
            else:
                # Fallback to raster
                row, col = raster.index(lon, lat)
                if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
                    vs30 = raster_array[row, col]
                else:
                    vs30 = np.nan
                measured = False
            vs30_values.append(vs30)
            measured_flags.append(measured)
        
        # Add columns to stations DataFrame
        df['Vs30'] = vs30_values
        df['measured'] = measured_flags
        sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                        'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                        'vs30measured': df['measured'], # true or false, not sure how this is used
                        'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                        'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                        }
        sitecollection = pd.DataFrame(sitecol_dict)
        sctx = SitesContext(sitecol=sitecollection)   
        
        ### Calculate Ground Motions
        
        # define metrics
        #IMT = imt.PGA() #IS THIS IN G OR %G
        IMT = imt.PGV()
        #IMT = imt.SA(period=0.1)
        uncertaintytype = const.StdDev.TOTAL
        
        # instantiate GMMs
        ASK14 = AbrahamsonEtAl2014()
        BSSA14 = BooreEtAl2014()
        CB14 = CampbellBozorgnia2014()
        CY14 = ChiouYoungs2014()
        
        # calculate ln ground motions for each
        ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        
        # average across all four for active crustal values
        ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
        ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1   
    
        ### Find Observed Data
        
        # gather observed ground motion metrics
        OBSdrup = df['RuptureDistance']
        OBSpga = df['PGA']*0.01 #%g
        OBSpgv = df['PGV'] #cm/s
        OBSsa = df['SA(T=1.0000, D=0.050)']
    
        ''' identify which metric is being worked with '''
        ObservedMetric = OBSpgv
        
        # calculate residuals with estimated data
        residuals = []
        for i in range(len(ln_median_ac14)):
            residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14[i] #normalized log residual
            residuals.append(residual)
        residuals = np.array(residuals)
        df['Residuals']= residuals
        mean_residual = np.mean(residuals)
        totalresiduals.append(mean_residual)
        mean_residual = round(mean_residual, 4)
    
        # (optional) find SNR values
        snr_df = pd.read_csv(f"/Users/carterdills/Documents/Reno1/{event}/data/{event}_default_snr.csv")
        # Prepare list to store the minimum SNRs for each StationID
        min_snrs = []
        # Iterate through each StationID
        for station_id in df['StationID']:
            # Find rows in snr_df where TraceID starts with the StationID
            matches = snr_df[snr_df['TraceID'].astype(str).str.startswith(str(station_id))]
        
            if not matches.empty:
                min_snr = matches['SNR(1)'].min()
            else:
                min_snr = float('0') # Or some default, like None or 0
                print('Did not find stationID in SNR csv')
            min_snrs.append(min_snr)
        
        # Add the result to the original DataFrame
        df['MinSNR'] = min_snrs
        
        ### Plot Model Results
        
        # figure setup
        fig, axi = plt.subplots(figsize=(6,4))
        axi.set_facecolor("gainsboro")
         # Create a colormap
        cmap = plt.colormaps['Reds']
        # Normalize the values to the range 0-1 for the colormap
        #norm = plt.Normalize(vmin=0, vmax=1520)
        norm = plt.Normalize(vmin=0, vmax=df['MinSNR'].max())
        
        # plot GMPE
        ly1 = ln_median_ac14 - ln_sd_ac14
        ly2 = ln_median_ac14 + ln_sd_ac14
        axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
        plt.scatter(OBSdrup, ObservedMetric, c=min_snrs, cmap=cmap, norm=norm, s=5, label=f"{event} Observed Motions", zorder=2)
        #plt.scatter(rrups,np.exp(ln_median_ac14),c=vs30_values, cmap='viridis', s=3)
        axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=0)
        #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
        #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
        
        # formatting
        plt.title(f'M{magnitude} {event}: residual={mean_residual}')
        axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
        if ObservedMetric.equals(OBSpgv):
            axi.set_ylabel("PGV [cm/s]",fontsize=16)
        if ObservedMetric.equals(OBSpga):
            axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
        if ObservedMetric.equals(OBSsa):
            axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
        axi.grid(lw=0.3)
        axi.legend(loc="upper right",fontsize=11)
        axi.set_xlim(9,320)
        plt.xscale('log')
        plt.yscale('log')
        plt.colorbar(label='Vs30')

        pdf_pages.savefig(fig)
        plt.close(fig)
    else:
        print('Outside magnitude')
# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
#plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
#plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')
pdf_pages.close()

In [ ]:
#SNR Analysis
 
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []

#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")
#eventdata = pd.read_csv("EventsPracticeSubset.csv")

#Prepare files for Vs30 analysis
# Load Vs30 CSV
vs30_df = pd.read_csv(f'/Users/carterdills/Documents/QGIS/RenoVS30.csv')
# Threshold distance in degrees
threshold_deg = 0.0001
# Build KDTree from vs30 points
tree_coords = vs30_df[['Latitude', 'Longitude']].to_numpy()
tree = cKDTree(tree_coords)
# Load raster
raster = rasterio.open(f'/Users/carterdills/Documents/vs30/vs30_mosaic.tif')
raster_array = raster.read(1)
transform = raster.transform

#pdf_pages = PdfPages('PGVoutputs.pdf')
station_snr_records = []
station_counts = {}       # Will hold {StationID: count}

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():

    '''#Comment this section out if you just want to use whole event dataset
    dt = datetime.fromisoformat(row['EarthquakeTime'])
    start = datetime.fromisoformat('2008-01-01T00:00:00.000000Z')
    end = datetime.fromisoformat('2009-01-01T00:00:00.000000Z')
    # Check if dt is between start and end
    if start <= dt <= end:'''
        
    event = row['eventID']
    magnitude = row['EarthquakeMagnitude']
    depth = row['EarthquakeDepth']
    
    # download more rupture data for the event
    cat = client.get_events(eventid=f"{event}")
    cat = cat[0]
    
    #find event specific data from gmprocess (Reno1)
    base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
    filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
    file_path = os.path.join(base_dir, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df = df.sort_values(by='RuptureDistance', ascending =True)
        df = df.reset_index(drop=True)
        
    ### Define Rupture Context
    
    rctx = RuptureContext()
    rctx.mag = magnitude

    #try to find focal mechanism to add rake and dip
    try:
        mech = cat.focal_mechanisms[0]
        nop = mech.nodal_planes['nodal_plane_1']
        rctx.rake = nop['rake']
        rctx.dip = nop['dip']
        print(f'Found a focal mechanism for {magnitude} {event}')
    except:
        rctx.rake = 0
        rctx.dip = 45
        
    # assume square ruptures with identical stress drop for every event
    moment = 10 ** (1.5*(magnitude+10.7))
    stressdrop = 3*10**6 #Pa
    L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
    L = L/1000 #km 
    rctx.ztor = depth-L/2 #from hypo to edge of square
    rctx.width =L #length of square
    rctx.hypo_depth = depth
    
    ### Define Distance Context (assume fault-perpendicular sites)
    
    dctx = DistancesContext()
    rjbs = df['JoynerBooreDistance'] # rjb values in km
    rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
    npts = rrups.size # number of distance points
    dctx.rrup = rrups # Closest distance to rupture surface
    dctx.rjb = rjbs # Distance to rupture’s surface projection
    dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
    dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                             #  (essentially, how far a location is "along the fault" from the rupture termination point)
    
    ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
    
    #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
    # Prepare results
    vs30_values = []
    measured_flags = []

    #Find Vs30 measurements from USGS raster
    # Iterate over stations
    for _, station in df.iterrows():
        lat = station['StationLatitude']
        lon = station['StationLongitude']
        # Query tree for neighbors within the threshold
        distances, indices = tree.query([[lat, lon]], k=1, distance_upper_bound=threshold_deg)
        index = indices[0]
        if index != tree.n:  # Found a valid neighbor
            vs30 = vs30_df.iloc[index]['Vs30']
            measured = True
        else:
            # Fallback to raster
            row, col = raster.index(lon, lat)
            if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
                vs30 = raster_array[row, col]
            else:
                vs30 = np.nan
            measured = False
        vs30_values.append(vs30)
        measured_flags.append(measured)
    
    # Add columns to stations DataFrame
    df['Vs30'] = vs30_values
    df['measured'] = measured_flags
    sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                    'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                    'vs30measured': df['measured'], # true or false, not sure how this is used
                    'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                    'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                    }
    sitecollection = pd.DataFrame(sitecol_dict)
    sctx = SitesContext(sitecol=sitecollection)   
    
    ### Calculate Ground Motions
    
    # define metrics
    #IMT = imt.PGA() #IS THIS IN G OR %G
    IMT = imt.PGV()
    #IMT = imt.SA(period=0.1)
    uncertaintytype = const.StdDev.TOTAL
    
    # instantiate GMMs
    ASK14 = AbrahamsonEtAl2014()
    BSSA14 = BooreEtAl2014()
    CB14 = CampbellBozorgnia2014()
    CY14 = ChiouYoungs2014()
    
    # calculate ln ground motions for each
    ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
    
    # average across all four for active crustal values
    ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
    ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1   

    ### Find Observed Data
    
    # gather observed ground motion metrics
    OBSdrup = df['RuptureDistance']
    OBSpga = df['PGA']*0.01 #%g
    OBSpgv = df['PGV'] #cm/s
    OBSsa = df['SA(T=1.0000, D=0.050)']

    ''' identify which metric is being worked with '''
    ObservedMetric = OBSpgv
    
    # calculate residuals with estimated data
    residuals = []
    for i in range(len(ln_median_ac14)):
        residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14[i] #normalized log residual
        residuals.append(residual)
    residuals = np.array(residuals)
    df['Residuals']= residuals
    mean_residual = np.mean(residuals)
    totalresiduals.append(mean_residual)
    mean_residual = round(mean_residual, 4)

    # (optional) find SNR values
    snr_df = pd.read_csv(f"/Users/carterdills/Documents/Reno1/{event}/data/{event}_default_snr.csv")
    # Prepare list to store the minimum SNRs for each StationID
    min_snrs = []
    # Iterate through each StationID
    for station_id in df['StationID']:
        station_id = str(station_id)  # Make sure it's a string
        # Update count for this StationID
        station_counts[station_id] = station_counts.get(station_id, 0) + 1
        # Find rows in snr_df where TraceID starts with the StationID
        matches = snr_df[snr_df['TraceID'].astype(str).str.startswith(str(station_id))]
    
        if not matches.empty:
            min_snr = matches['SNR(1)'].max()
        else:
            min_snr = float('0') # Or some default, like None or 0
            print('Did not find stationID in SNR csv')
        min_snrs.append(min_snr)
    
    # Add the result to the original DataFrame
    df['MinSNR'] = min_snrs
    station_snr_records.append({'StationID': station_id, 'MinSNR': min_snr})
    
    ### Plot Model Results
    
    # figure setup
    fig, axi = plt.subplots(figsize=(6,4))
    axi.set_facecolor("gainsboro")
     # Create a colormap
    cmap = plt.colormaps['Reds']
    # Normalize the values to the range 0-1 for the colormap
    #norm = plt.Normalize(vmin=0, vmax=1520)
    #norm = plt.Normalize(vmin=0, vmax=df['MinSNR'].min())
    norm = colors.LogNorm(vmin=0.27, vmax=df['MinSNR'].mean())
    
    # plot GMPE
    ly1 = ln_median_ac14 - ln_sd_ac14
    ly2 = ln_median_ac14 + ln_sd_ac14
    axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
    plt.scatter(OBSdrup, ObservedMetric, c=min_snrs, cmap=cmap, norm=norm, s=5, label=f"{event} Observed Motions", zorder=2)
    #plt.scatter(rrups,np.exp(ln_median_ac14),c=vs30_values, cmap='viridis', s=3)
    axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=0)
    #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
    #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
    
    # formatting
    plt.title(f'M{magnitude} {event}: residual={mean_residual}')
    axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
    if ObservedMetric.equals(OBSpgv):
        axi.set_ylabel("PGV [cm/s]",fontsize=16)
    if ObservedMetric.equals(OBSpga):
        axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
    if ObservedMetric.equals(OBSsa):
        axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
    axi.grid(lw=0.3)
    axi.legend(loc="upper right",fontsize=11)
    axi.set_xlim(9,320)
    plt.xscale('log')
    plt.yscale('log')
    plt.colorbar(label='SNR')

    #pdf_pages.savefig(fig)
    #plt.close(fig)

# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
#plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
#plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')
#pdf_pages.close()

snr_summary_df = pd.DataFrame(station_snr_records)
# Compute mean of minimum SNRs per station
mean_snr_per_station = snr_summary_df.groupby('StationID')['MinSNR'].mean().reset_index()
counts_df = pd.DataFrame(list(station_counts.items()), columns=['StationID', 'Count'])
# Merge counts into the summary
mean_snr_per_station = mean_snr_per_station.merge(counts_df, on='StationID')
mean_snr_per_station.rename(columns={'MinSNR': 'MeanMinSNR'}, inplace=True)
mean_snr_per_station = mean_snr_per_station.sort_values(by='MeanMinSNR', ascending=True)
# Print or save
pd.set_option('display.max_rows', None)
print(mean_snr_per_station)

In [ ]:
#SNR Analysis (Adding magnitude)
 
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []

#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")
#eventdata = pd.read_csv("EventsPracticeSubset.csv")

#Prepare files for Vs30 analysis
# Load Vs30 CSV
vs30_df = pd.read_csv(f'/Users/carterdills/Documents/QGIS/RenoVS30.csv')
# Threshold distance in degrees
threshold_deg = 0.0001
# Build KDTree from vs30 points
tree_coords = vs30_df[['Latitude', 'Longitude']].to_numpy()
tree = cKDTree(tree_coords)
# Load raster
raster = rasterio.open(f'/Users/carterdills/Documents/vs30/vs30_mosaic.tif')
raster_array = raster.read(1)
transform = raster.transform

#pdf_pages = PdfPages('PGVoutputs.pdf')
station_snr_records = []
station_counts = {}       # Will hold {StationID: count}

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():

    '''#Comment this section out if you just want to use whole event dataset
    dt = datetime.fromisoformat(row['EarthquakeTime'])
    start = datetime.fromisoformat('2008-01-01T00:00:00.000000Z')
    end = datetime.fromisoformat('2009-01-01T00:00:00.000000Z')
    # Check if dt is between start and end
    if start <= dt <= end:'''

    mag = row['EarthquakeMagnitude']
    minmag = 3
    maxmag = 5.7
    # Check if mag is between min and max
    if minmag <= mag <= maxmag:
        
        event = row['eventID']
        magnitude = row['EarthquakeMagnitude']
        depth = row['EarthquakeDepth']
        
        # download more rupture data for the event
        cat = client.get_events(eventid=f"{event}")
        cat = cat[0]
        
        #find event specific data from gmprocess (Reno1)
        base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
        filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
        file_path = os.path.join(base_dir, filename)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df = df.sort_values(by='RuptureDistance', ascending =True)
            df = df.reset_index(drop=True)
            
        ### Define Rupture Context
        
        rctx = RuptureContext()
        rctx.mag = magnitude
    
        #try to find focal mechanism to add rake and dip
        try:
            mech = cat.focal_mechanisms[0]
            nop = mech.nodal_planes['nodal_plane_1']
            rctx.rake = nop['rake']
            rctx.dip = nop['dip']
            print(f'Found a focal mechanism for {magnitude} {event}')
        except:
            rctx.rake = 0
            rctx.dip = 45
            
        # assume square ruptures with identical stress drop for every event
        moment = 10 ** (1.5*(magnitude+10.7))
        stressdrop = 3*10**6 #Pa
        L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
        L = L/1000 #km 
        rctx.ztor = depth-L/2 #from hypo to edge of square
        rctx.width =L #length of square
        rctx.hypo_depth = depth
        
        ### Define Distance Context (assume fault-perpendicular sites)
        
        dctx = DistancesContext()
        rjbs = df['JoynerBooreDistance'] # rjb values in km
        rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
        npts = rrups.size # number of distance points
        dctx.rrup = rrups # Closest distance to rupture surface
        dctx.rjb = rjbs # Distance to rupture’s surface projection
        dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
        dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                                 #  (essentially, how far a location is "along the fault" from the rupture termination point)
        
        ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
        
        #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
        # Prepare results
        vs30_values = []
        measured_flags = []
    
        #Find Vs30 measurements from USGS raster
        # Iterate over stations
        for _, station in df.iterrows():
            lat = station['StationLatitude']
            lon = station['StationLongitude']
            # Query tree for neighbors within the threshold
            distances, indices = tree.query([[lat, lon]], k=1, distance_upper_bound=threshold_deg)
            index = indices[0]
            if index != tree.n:  # Found a valid neighbor
                vs30 = vs30_df.iloc[index]['Vs30']
                measured = True
            else:
                # Fallback to raster
                row, col = raster.index(lon, lat)
                if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
                    vs30 = raster_array[row, col]
                else:
                    vs30 = np.nan
                measured = False
            vs30_values.append(vs30)
            measured_flags.append(measured)
        
        # Add columns to stations DataFrame
        df['Vs30'] = vs30_values
        df['measured'] = measured_flags
        sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                        'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                        'vs30measured': df['measured'], # true or false, not sure how this is used
                        'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                        'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                        }
        sitecollection = pd.DataFrame(sitecol_dict)
        sctx = SitesContext(sitecol=sitecollection)   
        
        ### Calculate Ground Motions
        
        # define metrics
        #IMT = imt.PGA() #IS THIS IN G OR %G
        IMT = imt.PGV()
        #IMT = imt.SA(period=0.1)
        uncertaintytype = const.StdDev.TOTAL
        
        # instantiate GMMs
        ASK14 = AbrahamsonEtAl2014()
        BSSA14 = BooreEtAl2014()
        CB14 = CampbellBozorgnia2014()
        CY14 = ChiouYoungs2014()
        
        # calculate ln ground motions for each
        ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        
        # average across all four for active crustal values
        ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
        ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1   
    
        ### Find Observed Data
        
        # gather observed ground motion metrics
        OBSdrup = df['RuptureDistance']
        OBSpga = df['PGA']*0.01 #%g
        OBSpgv = df['PGV'] #cm/s
        OBSsa = df['SA(T=1.0000, D=0.050)']
    
        ''' identify which metric is being worked with '''
        ObservedMetric = OBSpgv
        
        # calculate residuals with estimated data
        residuals = []
        for i in range(len(ln_median_ac14)):
            residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14[i] #normalized log residual
            residuals.append(residual)
        residuals = np.array(residuals)
        df['Residuals']= residuals
        mean_residual = np.mean(residuals)
        totalresiduals.append(mean_residual)
        mean_residual = round(mean_residual, 4)
    
        # (optional) find SNR values
        snr_df = pd.read_csv(f"/Users/carterdills/Documents/Reno1/{event}/data/{event}_default_snr.csv")
        # Prepare list to store the minimum SNRs for each StationID
        min_snrs = []
        # Iterate through each StationID
        for station_id in df['StationID']:
            station_id = str(station_id)  # Make sure it's a string
            # Update count for this StationID
            station_counts[station_id] = station_counts.get(station_id, 0) + 1
            # Find rows in snr_df where TraceID starts with the StationID
            matches = snr_df[snr_df['TraceID'].astype(str).str.startswith(str(station_id))]
        
            if not matches.empty:
                min_snr = matches['SNR(1)'].min()
            else:
                min_snr = float('0') # Or some default, like None or 0
                print('Did not find stationID in SNR csv')
            min_snrs.append(min_snr)
        
        # Add the result to the original DataFrame
        df['MinSNR'] = min_snrs
        station_snr_records.append({'StationID': station_id, 'MinSNR': min_snr})
        
        ### Plot Model Results
        
        # figure setup
        fig, axi = plt.subplots(figsize=(6,4))
        axi.set_facecolor("gainsboro")
         # Create a colormap
        cmap = plt.colormaps['Reds']
        # Normalize the values to the range 0-1 for the colormap
        #norm = plt.Normalize(vmin=0, vmax=1520)
        #norm = plt.Normalize(vmin=0, vmax=df['MinSNR'].min())
        norm = colors.LogNorm(vmin=0.27, vmax=df['MinSNR'].mean())
        
        # plot GMPE
        ly1 = ln_median_ac14 - ln_sd_ac14
        ly2 = ln_median_ac14 + ln_sd_ac14
        axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
        plt.scatter(OBSdrup, ObservedMetric, c=min_snrs, cmap=cmap, norm=norm, s=5, label=f"{event} Observed Motions", zorder=2)
        #plt.scatter(rrups,np.exp(ln_median_ac14),c=vs30_values, cmap='viridis', s=3)
        axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=0)
        #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
        #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
        
        # formatting
        plt.title(f'M{magnitude} {event}: residual={mean_residual}')
        axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
        if ObservedMetric.equals(OBSpgv):
            axi.set_ylabel("PGV [cm/s]",fontsize=16)
        if ObservedMetric.equals(OBSpga):
            axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
        if ObservedMetric.equals(OBSsa):
            axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
        axi.grid(lw=0.3)
        axi.legend(loc="upper right",fontsize=11)
        axi.set_xlim(9,320)
        plt.xscale('log')
        plt.yscale('log')
        plt.colorbar(label='SNR')
    
        #pdf_pages.savefig(fig)
        #plt.close(fig)
    else:
        print('Out of magnitude range')

# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
#plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
#plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')
#pdf_pages.close()

snr_summary_df = pd.DataFrame(station_snr_records)
# Compute mean of minimum SNRs per station
mean_snr_per_station = snr_summary_df.groupby('StationID')['MinSNR'].mean().reset_index()
counts_df = pd.DataFrame(list(station_counts.items()), columns=['StationID', 'Count'])
# Merge counts into the summary
mean_snr_per_station = mean_snr_per_station.merge(counts_df, on='StationID')
mean_snr_per_station.rename(columns={'MinSNR': 'MeanMinSNR'}, inplace=True)
mean_snr_per_station = mean_snr_per_station.sort_values(by='MeanMinSNR', ascending=True)
# Print or save
pd.set_option('display.max_rows', None)
print(mean_snr_per_station)

In [ ]:
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []
totalmagnitudes = []
#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")
#eventdata = pd.read_csv("EventsPracticeSubset.csv")

#Prepare files for Vs30 analysis
# Load Vs30 CSV
vs30_df = pd.read_csv(f'/Users/carterdills/Documents/QGIS/RenoVS30.csv')
# Threshold distance in degrees
threshold_deg = 0.0001
# Build KDTree from vs30 points
tree_coords = vs30_df[['Latitude', 'Longitude']].to_numpy()
tree = cKDTree(tree_coords)
# Load raster
raster = rasterio.open(f'/Users/carterdills/Documents/vs30/vs30_mosaic.tif')
raster_array = raster.read(1)
transform = raster.transform

#pdf_pages = PdfPages('PGVoutputs.pdf')
station_snr_records = []
station_counts = {}       # Will hold {StationID: count}

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():

    '''#Comment this section out if you just want to use whole timeline
    dt = datetime.fromisoformat(row['EarthquakeTime'])
    start = datetime.fromisoformat('2008-01-01T00:00:00.000000Z')
    end = datetime.fromisoformat('2009-01-01T00:00:00.000000Z')
    # Check if dt is between start and end
    if start <= dt <= end:'''

    mag = row['EarthquakeMagnitude']
    minmag = 3.5
    maxmag = 4
    # Check if mag is between min and max
    if minmag <= mag <= maxmag:
        
        event = row['eventID']
        magnitude = row['EarthquakeMagnitude']
        depth = row['EarthquakeDepth']
        totalmagnitudes.append(magnitude)
        # download more rupture data for the event
        cat = client.get_events(eventid=f"{event}")
        cat = cat[0]
        
        #find event specific data from gmprocess (Reno1)
        base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
        filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
        file_path = os.path.join(base_dir, filename)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df = df.sort_values(by='RuptureDistance', ascending =True)
            df = df.reset_index(drop=True)
            
        ### Define Rupture Context
        
        rctx = RuptureContext()
        rctx.mag = magnitude
    
        #try to find focal mechanism to add rake and dip
        try:
            mech = cat.focal_mechanisms[0]
            nop = mech.nodal_planes['nodal_plane_1']
            rctx.rake = nop['rake']
            rctx.dip = nop['dip']
            print(f'Found a focal mechanism for {magnitude} {event}')
        except:
            rctx.rake = 0
            rctx.dip = 45
            
        # assume square ruptures with identical stress drop for every event
        moment = 10 ** (1.5*(magnitude+10.7))
        stressdrop = 10*10**6 #Pa
        L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
        L = L/1000 #km 
        rctx.ztor = depth-L/2 #from hypo to edge of square
        rctx.width =L #length of square
        rctx.hypo_depth = depth
        
        ### Define Distance Context (assume fault-perpendicular sites)
        
        dctx = DistancesContext()
        rjbs = df['JoynerBooreDistance'] # rjb values in km
        rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
        npts = rrups.size # number of distance points
        dctx.rrup = rrups # Closest distance to rupture surface
        dctx.rjb = rjbs # Distance to rupture’s surface projection
        dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
        dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                                 #  (essentially, how far a location is "along the fault" from the rupture termination point)
        
        ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
        
        #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
        # Prepare results
        vs30_values = []
        measured_flags = []
    
        #Find Vs30 measurements from USGS raster
        # Iterate over stations
        for _, station in df.iterrows():
            lat = station['StationLatitude']
            lon = station['StationLongitude']
            # Query tree for neighbors within the threshold
            distances, indices = tree.query([[lat, lon]], k=1, distance_upper_bound=threshold_deg)
            index = indices[0]
            if index != tree.n:  # Found a valid neighbor
                vs30 = vs30_df.iloc[index]['Vs30']
                measured = True
            else:
                # Fallback to raster
                row, col = raster.index(lon, lat)
                if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
                    vs30 = raster_array[row, col]
                else:
                    vs30 = np.nan
                measured = False
            vs30_values.append(vs30)
            measured_flags.append(measured)
        
        # Add columns to stations DataFrame
        df['Vs30'] = vs30_values
        df['measured'] = measured_flags
        sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                        'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s
                        'vs30measured': df['measured'], # true or false, not sure how this is used
                        'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                        'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                        }
        sitecollection = pd.DataFrame(sitecol_dict)
        sctx = SitesContext(sitecol=sitecollection)   
        
        ### Calculate Ground Motions
        
        # define metrics
        #IMT = imt.PGA() #IS THIS IN G OR %G
        IMT = imt.PGV()
        #IMT = imt.SA(period=0.1)
        uncertaintytype = const.StdDev.TOTAL
        
        # instantiate GMMs
        ASK14 = AbrahamsonEtAl2014()
        BSSA14 = BooreEtAl2014()
        CB14 = CampbellBozorgnia2014()
        CY14 = ChiouYoungs2014()
        
        # calculate ln ground motions for each
        ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
        
        # average across all four for active crustal values
        ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
        ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1   
    
        ### Find Observed Data
        
        # gather observed ground motion metrics
        OBSdrup = df['RuptureDistance']
        OBSpga = df['PGA']*0.01 #%g
        OBSpgv = df['PGV'] #cm/s
        OBSsa = df['SA(T=1.0000, D=0.050)']
    
        ''' identify which metric is being worked with '''
        ObservedMetric = OBSpgv
        
        # calculate residuals with estimated data
        residuals = []
        for i in range(len(ln_median_ac14)):
            residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14[i] #normalized log residual
            residuals.append(residual)
        residuals = np.array(residuals)
        df['Residuals']= residuals
        mean_residual = np.mean(residuals)
        totalresiduals.append(mean_residual)
        mean_residual = round(mean_residual, 4)
    
        # (optional) find SNR values
        snr_df = pd.read_csv(f"/Users/carterdills/Documents/Reno1/{event}/data/{event}_default_snr.csv")
        # Prepare list to store the minimum SNRs for each StationID
        min_snrs = []
        # Iterate through each StationID
        for station_id in df['StationID']:
            station_id = str(station_id)  # Make sure it's a string
            # Update count for this StationID
            station_counts[station_id] = station_counts.get(station_id, 0) + 1
            # Find rows in snr_df where TraceID starts with the StationID
            matches = snr_df[snr_df['TraceID'].astype(str).str.startswith(str(station_id))]
        
            if not matches.empty:
                min_snr = matches['SNR(1)'].min()
            else:
                min_snr = float('0') # Or some default, like None or 0
                print('Did not find stationID in SNR csv')
            min_snrs.append(min_snr)
        
        # Add the result to the original DataFrame
        df['MinSNR'] = min_snrs
        station_snr_records.append({'StationID': station_id, 'MinSNR': min_snr})
        
        ### Plot Model Results
        
        # figure setup
        fig, axi = plt.subplots(figsize=(6,4))
        axi.set_facecolor("gainsboro")
         # Create a colormap
        cmap = plt.colormaps['Reds']
        # Normalize the values to the range 0-1 for the colormap
        #norm = plt.Normalize(vmin=0, vmax=1520)
        #norm = plt.Normalize(vmin=0, vmax=df['MinSNR'].min())
        norm = colors.LogNorm(vmin=0.27, vmax=df['MinSNR'].mean())
        
        # plot GMPE
        ly1 = ln_median_ac14 - ln_sd_ac14
        ly2 = ln_median_ac14 + ln_sd_ac14
        axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
        plt.scatter(OBSdrup, ObservedMetric, c=min_snrs, cmap=cmap, norm=norm, s=5, label=f"{event} Observed Motions", zorder=2)
        #plt.scatter(rrups,np.exp(ln_median_ac14),c=vs30_values, cmap='viridis', s=3)
        axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=0)
        #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
        #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
        
        # formatting
        plt.title(f'M{magnitude} {event}: residual={mean_residual}')
        axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
        if ObservedMetric.equals(OBSpgv):
            axi.set_ylabel("PGV [cm/s]",fontsize=16)
        if ObservedMetric.equals(OBSpga):
            axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
        if ObservedMetric.equals(OBSsa):
            axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
        axi.grid(lw=0.3)
        axi.legend(loc="upper right",fontsize=11)
        axi.set_xlim(9,320)
        plt.xscale('log')
        plt.yscale('log')
        plt.colorbar(label='SNR')
    
        #pdf_pages.savefig(fig)
        #plt.close(fig)
    else:
        print('Out of magnitude range')

# Save the figure with a specified DPI
#plt.savefig("ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
#plt.show()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
#plt.savefig("ResidualsHistograms.png", dpi=300, bbox_inches='tight')
#pdf_pages.close()

snr_summary_df = pd.DataFrame(station_snr_records)
# Compute mean of minimum SNRs per station
mean_snr_per_station = snr_summary_df.groupby('StationID')['MinSNR'].mean().reset_index()
counts_df = pd.DataFrame(list(station_counts.items()), columns=['StationID', 'Count'])
# Merge counts into the summary
mean_snr_per_station = mean_snr_per_station.merge(counts_df, on='StationID')
mean_snr_per_station.rename(columns={'MinSNR': 'MeanMinSNR'}, inplace=True)
mean_snr_per_station = mean_snr_per_station.sort_values(by='MeanMinSNR', ascending=True)
# Print or save
pd.set_option('display.max_rows', None)
print(mean_snr_per_station)

In [ ]:
### Setup for Calculations

#initialize list to store residuals for each event
totalresiduals = []
totalmagnitudes = []
totaldistances = []
totalstationresiduals = []
allSNRrecordings = pd.DataFrame()
#setup client for rupture geometry data
client = Client("USGS")

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")
#eventdata = pd.read_csv("ParkerButteTestCSV.csv")
#eventdata = pd.read_csv("EventsPracticeSubset.csv")

#Prepare files for Vs30 analysis
# Load Vs30 CSV
vs30_df = pd.read_csv(f'/Users/carterdills/Documents/QGIS/RenoVS30.csv')
# Threshold distance in degrees
threshold_deg = 0.0001
# Build KDTree from vs30 points
tree_coords = vs30_df[['Latitude', 'Longitude']].to_numpy()
tree = cKDTree(tree_coords)
# Load raster
raster = rasterio.open(f'/Users/carterdills/Documents/vs30/vs30_mosaic.tif')
raster_array = raster.read(1)
transform = raster.transform

pdf_pages = PdfPages('/Users/carterdills/openquake/lib/python3.11/site-packages/OpenquakeFigures/PGVoutputs.pdf')
station_snr_records = []
station_counts = {}       # Will hold {StationID: count}

#loop through events to plot gm metrics for each
for _, row in eventdata.iterrows():

    #Comment this section out if you just want to use whole event dataset
    dt = datetime.fromisoformat(row['EarthquakeTime'])
    start = datetime.fromisoformat('2000-01-01T00:00:00.000000Z')
    end = datetime.fromisoformat('2025-06-01T00:00:00.000000Z')
    # Check if dt is between start and end
    if start <= dt <= end:

        mag = row['EarthquakeMagnitude']
        minmag = 2.5
        maxmag = 5.7
        # Check if mag is between min and max
        if minmag <= mag <= maxmag:
            
            event = row['eventID']
            magnitude = row['EarthquakeMagnitude']
            depth = row['EarthquakeDepth']
            totalmagnitudes.append(magnitude)
            # download more rupture data for the event
            cat = client.get_events(eventid=f"{event}")
            cat = cat[0]
            
            #find event specific data from gmprocess (Reno1)
            base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
            filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
            file_path = os.path.join(base_dir, filename)
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                df = df.sort_values(by='RuptureDistance', ascending =True)
                df = df.reset_index(drop=True)

            ### Determine Magnitude Type
            try:
                value = df.iat[1, 6]
                if pd.notna(value):
                    magnitudetype = value.strip().lower()
                else:
                    raise ValueError
            except (IndexError, ValueError, AttributeError):
                print(f"Didn't find magnitudetype for {event}")
                magnitudetype = "mw"

            if magnitudetype != "mw":
                try:
                    nil_df = pd.read_csv(f'/Users/carterdills/Documents/nsl_webmt.csv')
                    event_suffix = str(event)[-6:]
                    if event_suffix in nil_df.iloc[:, 0].astype(str).values:
                        print(f"Replaced {magnitudetype} with mw for {event}")
                        magnitudetype = "mw"
                    else:
                        print(f"Converted {magnitudetype} for {event} mathematically")

                        x = magnitude  # store current value
                        if magnitudetype == "ml":
                            if 3.5 <= x <= 7:
                                magnitude = 2.29155 + 0.02207 * x + 0.09782 * x**2
                            elif x < 3.5:
                                magnitude = 1.09329 + 0.70679 * x
                            else:
                                print(f"No magnitude adjustment could be made for {event}")
                        elif magnitudetype == "md":
                            if 3.5 <= x <= 7:
                                magnitude = 2.34570 + 0.00448 * x + 0.10308 * x**2
                            elif x < 3.5:
                                magnitude = 1.08291 + 0.72608 * x
                            else:
                                print(f"No magnitude adjustment could be made for {event}")
                        else:
                            print(f"No magnitude adjustment could be made for {event}")
                except Exception as e:
                    print(f"Error reading nil_webmt.csv or processing event {event}: {e}")
                    print(f"Assuming mathematical conversion for {magnitudetype} on {event}")
                    
                    x = magnitude
                    if magnitudetype == "ml":
                        if 3.5 <= x <= 7:
                            magnitude = 2.29155 + 0.02207 * x + 0.09782 * x**2
                        elif x < 3.5:
                            magnitude = 1.09329 + 0.70679 * x
                        else:
                            print(f"No magnitude adjustment could be made for {event}")
                    elif magnitudetype == "md":
                        if 3.5 <= x <= 7:
                            magnitude = 2.34570 + 0.00448 * x + 0.10308 * x**2
                        elif x < 3.5:
                            magnitude = 1.08291 + 0.72608 * x
                        else:
                            print(f"No magnitude adjustment could be made for {event}")
                    else:
                        print(f"No magnitude adjustment could be made for {event}")

            ### Define Rupture Context
            
            rctx = RuptureContext()
            rctx.mag = magnitude
        
            #try to find focal mechanism to add rake and dip
            try:
                mech = cat.focal_mechanisms[0]
                nop = mech.nodal_planes['nodal_plane_1']
                rctx.rake = nop['rake']
                rctx.dip = nop['dip']
                print(f'Found a focal mechanism for {magnitude} {event}')
            except:
                rctx.rake = 0
                rctx.dip = 45
                print(f'No focal mechanism available for {magnitude} {event}')
                
            # assume square ruptures with identical stress drop for every event
            moment = 10 ** (1.5*(magnitude+10.7))
            moment = moment * 10**-7
            stressdrop = 3*10**6 #Pa
            L = ((3*np.pi*moment)/(16*stressdrop))**(1/3) #m
            L = L/1000 #km 
            rctx.ztor = depth-L/2 #from hypo to edge of square
            rctx.width = L #length of square
            rctx.hypo_depth = depth
            
            ### Define Distance Context (assume fault-perpendicular sites)
            
            dctx = DistancesContext()
            rjbs = df['JoynerBooreDistance'] # rjb values in km
            rrups = df['RuptureDistance'] # calculated rupture distance for a vertical fault
            npts = rrups.size # number of distance points
            dctx.rrup = rrups # Closest distance to rupture surface
            dctx.rjb = rjbs # Distance to rupture’s surface projection
            dctx.rx = np.zeros(npts) # Perpendicular distance to rupture top edge projection
            dctx.ry0 = np.zeros(npts) # Horizontal distance off the end of the rupture measured parallel to strike 
                                     #  (essentially, how far a location is "along the fault" from the rupture termination point)
            
            distancestostore = df['RuptureDistance'].tolist()
            totaldistances.extend(distancestostore)
            
            ### Site context - seems to now need to be from a "site collection", which seems to be a pandas dataframe.
            
            #    value to predict - if you're doing it in array form (many records for a single earthquake), then make these arrays.
            # Prepare results
            vs30_values = []
            measured_flags = []
        
            #Find Vs30 measurements from USGS raster
            # Iterate over stations
            for _, station in df.iterrows():
                lat = station['StationLatitude']
                lon = station['StationLongitude']
                # Query tree for neighbors within the threshold
                distances, indices = tree.query([[lat, lon]], k=1, distance_upper_bound=threshold_deg)
                index = indices[0]
                if index != tree.n:  # Found a valid neighbor
                    vs30 = vs30_df.iloc[index]['Vs30']
                    measured = True
                else:
                    # Fallback to raster
                    row, col = raster.index(lon, lat)
                    if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
                        vs30 = raster_array[row, col]
                    else:
                        vs30 = np.nan
                    measured = False
                vs30_values.append(vs30)
                measured_flags.append(measured)
            
            # Add columns to stations DataFrame
            df['Vs30'] = vs30_values
            df['measured'] = measured_flags
            sitecol_dict = {'sids': np.arange(1, npts + 1), # site id
                            'vs30': vs30_values, # Vs30 value in m/s
                            'vs30measured': df['measured'], # true or false, not sure how this is used
                            'z1pt0': 50.0 + np.zeros(npts, dtype=float), # depth in m (not km) to 1.0 km/s horizon
                            'z2pt5': np.nan + np.zeros(npts, dtype=float), # depth in km (not m) to 2.5 km/s horizon
                            }
            """'vs30': 760.0 + np.zeros(npts, dtype=float), # Vs30 value in m/s"""
            sitecollection = pd.DataFrame(sitecol_dict)
            sctx = SitesContext(sitecol=sitecollection)   
            
            ### Calculate Ground Motions
            
            # define metrics
            #IMT = imt.PGA() #IS THIS IN G OR %G
            IMT = imt.PGV()
            #IMT = imt.SA(period=0.1)
            uncertaintytype = const.StdDev.TOTAL
            
            # instantiate GMMs
            ASK14 = AbrahamsonEtAl2014()
            BSSA14 = BooreEtAl2014()
            CB14 = CampbellBozorgnia2014()
            CY14 = ChiouYoungs2014()
            
            # calculate ln ground motions for each
            ln_median_ask14, ln_sd_ask14 = ASK14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
            ln_median_bssa14, ln_sd_bssa14 = BSSA14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
            ln_median_cb14, ln_sd_cb14 = CB14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
            ln_median_cy14, ln_sd_cy14 = CY14.get_mean_and_stddevs(sctx, rctx, dctx, IMT, [uncertaintytype])
            
            # average across all four for active crustal values
            ln_median_ac14 = 0.25 * (ln_median_ask14 + ln_median_bssa14 + ln_median_cb14 + ln_median_cy14)
            ln_sd_ac14 = 0.25 * (ln_sd_ask14[0] + ln_sd_bssa14[0] + ln_sd_cb14[0] + ln_sd_cy14[0]) # sds are in a list of length 1   
        
            ### Find Observed Data
            
            # gather observed ground motion metrics
            OBSdrup = df['RuptureDistance']
            OBSpga = df['PGA']*0.01 #%g
            OBSpgv = df['PGV'] #cm/s
            OBSsa = df['SA(T=1.0000, D=0.050)']
        
            ''' identify which metric is being worked with '''
            ObservedMetric = OBSpgv
            
            # calculate residuals with estimated data
            residuals = []
            for i in range(len(ln_median_ac14)):
                residual = (np.log(ObservedMetric[i])-(ln_median_ac14[i]))/ln_sd_ac14[i] #normalized log residual
                residuals.append(residual)
            residuals = np.array(residuals)
            df['Residuals']= residuals
            mean_residual = np.mean(residuals)
            totalresiduals.append(mean_residual)
            mean_residual = round(mean_residual, 4)
            totalstationresiduals.extend(residuals)
        
            # Find SNR values
            snr_df = pd.read_csv(f"/Users/carterdills/Documents/Reno1/{event}/data/{event}_default_snr.csv")
            # Prepare list to store the minimum SNRs for each StationID
            min_snrs = []
            # Iterate through each StationID
            for station_id in df['StationID']:
                station_id = str(station_id)  # Make sure it's a string
                # Update count for this StationID
                station_counts[station_id] = station_counts.get(station_id, 0) + 1
                # Find rows in snr_df where TraceID starts with the StationID
                matches = snr_df[
                    snr_df['TraceID'].astype(str).str.startswith(str(station_id)) &
                    ~snr_df['TraceID'].astype(str).str.endswith('Z')
                ]
            
                if not matches.empty:
                    min_snr = matches['SNR(1)'].min()
                    # Add the result to the original DataFrame
                    min_snrs.append(min_snr)
                    station_snr_records.append({'StationID': station_id, 'MinSNR': min_snr})
                
                else:
                    min_snr = float('0') # Or some default, like None or 0
                    print('Did not find stationID in SNR csv')
                
            df['MinSNR'] = min_snrs
                
            ### Plot Model Results
            
            # figure setup
            fig, axi = plt.subplots(figsize=(6,4))
            axi.set_facecolor("gainsboro")
            # Create a colormap
            cmap = plt.colormaps['Reds']
            # Normalize the values to the range 0-1 for the colormap
            #norm = plt.Normalize(vmin=0, vmax=1520)
            #norm = plt.Normalize(vmin=0, vmax=df['MinSNR'].min())
            norm = colors.LogNorm(vmin=0.27, vmax=df['MinSNR'].mean())
            
            # plot GMPE
            ly1 = ln_median_ac14 - ln_sd_ac14
            ly2 = ln_median_ac14 + ln_sd_ac14
            axi.fill_between(x=rrups,y1=np.exp(ly1),y2=np.exp(ly2),color="r",alpha=0.25,lw=2)
            plt.scatter(OBSdrup, ObservedMetric, c=min_snrs, cmap=cmap, norm=norm, s=5, label=f"{event} Observed Motions", zorder=2)
            #plt.scatter(rrups,np.exp(ln_median_ac14),c=vs30_values, cmap='viridis', s=3)
            axi.loglog(rrups,np.exp(ln_median_ac14),"-r",lw=3,label="Active Crustal GMM ($V_{S30}$=760m/s)", zorder=0)
            #axi.fill_between(x=rrups,y1=np.exp(ly1)+0.01,y2=np.exp(ly2)+0.05,color="b",alpha=0.25,lw=2)
            #axi.loglog(rrups,np.exp(ln_median_ac14)+0.05,"-b",lw=3,label="Hypothetical Updated GMM", zorder=1)
            
            # formatting
            plt.title(f'M{magnitude} {event}: residual={mean_residual}')
            axi.set_xlabel("$R_{rup}$ [km]",fontsize=16)
            if ObservedMetric.equals(OBSpgv):
                axi.set_ylabel("PGV [cm/s]",fontsize=16)
            if ObservedMetric.equals(OBSpga):
                axi.set_ylabel("PGA [%g]", fontsize=16)    ###Is something wrong with PGA units? Both should be %g but off by 100x, maybe predictions are just g
            if ObservedMetric.equals(OBSsa):
                axi.set_ylabel("SA (T=1.0000, D=0.050)[add units]", fontsize=12)
            axi.grid(lw=0.3)
            axi.legend(loc="upper right",fontsize=11)
            axi.set_xlim(9,320)
            plt.xscale('log')
            plt.yscale('log')
            plt.colorbar(label='SNR')
        
            pdf_pages.savefig(fig)
            plt.close(fig)
        else:
            print('Outside Magnitude Range')
    else:
        print('Outside Date Range')
# Save the figure with a specified DPI
#plt.savefig("/OpenquakeFigures/ProposalGMMFigure.png", dpi=300, bbox_inches='tight')

# show all gmm figures
#plt.show()
pdf_pages.close()

#plot residual distribution
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].hist(totalresiduals, bins=20, edgecolor = "black")
axes[0].set_title('Residuals Histogram (20 Bins)')
axes[0].set_xlabel('Mean Residual Difference')
axes[0].set_ylabel('Frequency')

M = math.ceil(max(totalresiduals))
m = math.floor(min(totalresiduals))
binwidth = 1
numberbins = np.arange(m, M + binwidth, binwidth)

axes[1].hist(totalresiduals, bins=numberbins, edgecolor = "black")
axes[1].set_title(f'Residuals Histogram ({len(numberbins)} Bins Integers)')
axes[1].set_xlabel('Mean Residual Difference')
axes[1].set_ylabel('Frequency')
plt.savefig("/Users/carterdills/openquake/lib/python3.11/site-packages/OpenquakeFigures/ResidualsHistograms.png", dpi=300, bbox_inches='tight')

snr_summary_df = pd.DataFrame(station_snr_records)
# Compute mean of minimum SNRs per station
mean_snr_per_station = snr_summary_df.groupby('StationID')['MinSNR'].mean().reset_index()
counts_df = pd.DataFrame(list(station_counts.items()), columns=['StationID', 'Count'])
# Merge counts into the summary
mean_snr_per_station = mean_snr_per_station.merge(counts_df, on='StationID')
mean_snr_per_station.rename(columns={'MinSNR': 'MeanMinSNR'}, inplace=True)
mean_snr_per_station = mean_snr_per_station.sort_values(by='MeanMinSNR', ascending=True)
# Print or save
pd.set_option('display.max_rows', None)
print(mean_snr_per_station)

fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].scatter(totalmagnitudes, totalresiduals, s=2)
axes[0].set_title('Magnitude vs Event Residual')
axes[0].set_xlabel('Magnitude')
axes[0].set_ylabel('Mean Residual')

axes[1].scatter(totaldistances, totalstationresiduals, s=1)
axes[1].set_title('Distance vs Station Residual')
axes[1].set_xlabel('Rrup (km)')
axes[1].set_ylabel('Station Residual Difference')
plt.savefig("/Users/carterdills/openquake/lib/python3.11/site-packages/OpenquakeFigures/maganddistwithresiduals.png", dpi=300, bbox_inches='tight')

In [6]:
#COUNT RECORDINGS PER EVENT

# Load event IDs from CSV
eventdata = pd.read_csv("ListofCompletedEvents.csv")
event_ids = eventdata['eventID']

# Initialize list to store results
row_counts = []

# Loop through each event
for event in event_ids:
    base_dir = f"/Users/carterdills/Documents/Reno1/{event}/data"
    filename = f"{event}_default_metrics_rotd(percentile=50.0).csv"
    file_path = os.path.join(base_dir, filename)

    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            row_count = len(df)  # includes data rows only
        except Exception as e:
            print(f"Failed to read {file_path}: {e}")
            row_count = -1  # flag for read error
    else:
        print(f"File not found: {file_path}")
        row_count = 0  # file missing

    # Store result
    row_counts.append({'eventID': event, 'NumRows': row_count})

# Save results to CSV
counts_df = pd.DataFrame(row_counts)
counts_df = counts_df.sort_values(by='NumRows', ascending=True)

# Print summary
# Set display options to show all rows, columns, and full column width
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None) # Or a large integer for max characters
print(counts_df)
# Optionally, reset display options to default
pd.reset_option('all')

        eventID  NumRows
117  nn00278760        1
111  nn00278758        1
106  nn00278756        1
393  nn00888664        1
300  nn00278754        1
299  nn00278762        2
410  nc71126849        3
156  nn00185054        3
318  nn00001802        4
180  nn00487790        4
127  nn00002400        5
406  nn00345789        5
401  nc21152723        5
370  nn00208650        5
158  nn00191064        6
155  nc21169351        7
61   nc72796075        7
198  nn00001757        7
125  nn00436637        8
224  nn00651540        8
337  nn00278764        8
46   nn00059455        9
227  nc21105736       10
9    nc21227205       10
448  nn00167838       10
313  nc21142104       11
414  nc21126268       11
261  nc21099303       12
409  nc21130131       12
433  nn00020076       12
427  nn00021354       12
274  nn00178631       13
270  nc21254752       13
141  nc21086071       13
398  nn00279090       14
269  nn00106671       15
129  nc21267688       15
454  nn00518396       15
371  nn00123032       16


In [ ]:
#SNR Ratio Assembly

#load basic event characteristic data (query2)
eventdata = pd.read_csv("ListofCompletedEvents.csv")
#eventdata = pd.read_csv("ParkerButteTestCSV.csv")
#eventdata = pd.read_csv("EventsPracticeSubset.csv")

for _, row in eventdata.iterrows():
    event = row['eventID']
    # Find SNR values
            snr_df = pd.read_csv(f"/Users/carterdills/Documents/Reno1/{event}/data/{event}_default_snr.csv")
            # Prepare list to store the minimum SNRs for each StationID
            min_snrs = []
            # Iterate through each StationID
            for station_id in df['StationID']:
                station_id = str(station_id)  # Make sure it's a string
                # Update count for this StationID
                station_counts[station_id] = station_counts.get(station_id, 0) + 1
                # Find rows in snr_df where TraceID starts with the StationID
                matches = snr_df[
                    snr_df['TraceID'].astype(str).str.startswith(str(station_id)) &
                    ~snr_df['TraceID'].astype(str).str.endswith('Z')
                ]
            
                if not matches.empty:
                    min_snr = matches['SNR(1)'].min()
                    # Add the result to the original DataFrame
                    min_snrs.append(min_snr)
                    station_snr_records.append({'StationID': station_id, 'MinSNR': min_snr})
                
                else:
                    min_snr = float('0') # Or some default, like None or 0
                    print('Did not find stationID in SNR csv')
                
            df['MinSNR'] = min_snrs


In [2]:
# Load event list
events_df = pd.read_csv("ListofCompletedEvents.csv")
event_ids = events_df['eventID'].dropna().astype(str).tolist()

# Thresholds and bins
thresholds = [1, 2, 3, 4, 5, 8, 10, 20]
magnitude_bins = [(2.5, 3), (3, 4), (4, 5), (5, 6)]

# Storage
all_data = []  # Store all SNR rows with their magnitude and EarthquakeId

# Stats
missing_files = 0
events_processed = 0

for event in event_ids:
    snr_path = f"/Users/carterdills/Documents/Reno1/{event}/data/{event}_default_snr.csv"

    if not os.path.exists(snr_path):
        print(f"Missing file: {event}")
        missing_files += 1
        continue

    try:
        df = pd.read_csv(snr_path)
        if 'EarthquakeMagnitude' not in df.columns:
            print(f"Missing magnitude column in {event}")
            continue

        snr_columns = df.columns[-16:]
        mag = df['EarthquakeMagnitude'].values[0]
        snr_df = df[snr_columns].copy()
        snr_df['Magnitude'] = mag
        snr_df['EarthquakeId'] = df['EarthquakeId']

        all_data.append(snr_df)
        events_processed += 1

    except Exception as e:
        print(f"Error in {event}: {e}")

# Combine all data into one DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Frequencies from SNR columns
frequency_labels = combined_df.columns[:-2].tolist()  # exclude 'Magnitude' and 'EarthquakeId'

# Function to calculate failure matrix
def calculate_failure_matrix(data, thresholds, freq_labels):
    failure_counts = pd.DataFrame(0, index=freq_labels, columns=thresholds)
    total = data.shape[0]

    for threshold in thresholds:
        failures = (data[freq_labels] < threshold).sum()
        failure_counts[threshold] = failures

    return failure_counts, total

# Function to plot heatmap with annotations
def plot_heatmap(failure_counts, total, n_events, title_prefix):
    heatmap_data = failure_counts.astype(int).T
    annotations = heatmap_data.copy().astype(str)

    for t in heatmap_data.index:
        for f in heatmap_data.columns:
            count = heatmap_data.at[t, f]
            percent = (count / total) * 100 if total > 0 else 0
            annotations.at[t, f] = f"{count} ({percent:.1f}%)"

    plt.figure(figsize=(12, 8))
    sns.heatmap(
        heatmap_data.T,
        annot=annotations.T,
        annot_kws={"size": 9},
        fmt="",
        cmap="Reds",
        cbar_kws={'label': 'Count of Failing Recordings'}
    )
    plt.title(f"{title_prefix} [{n_events} Events, {total} Recordings]")
    plt.xlabel("SNR Threshold")
    plt.ylabel("Frequency (Hz)")
    plt.tight_layout()

# Save all plots into one PDF
with PdfPages("/Users/carterdills/openquake/lib/python3.11/site-packages/OpenquakeFigures/SNR_heatmaps.pdf") as pdf:
    # All data
    failures_all, total_all = calculate_failure_matrix(combined_df, thresholds, frequency_labels)
    n_events_all = combined_df['EarthquakeId'].nunique()
    plot_heatmap(failures_all, total_all, n_events_all, "SNR Failures (All Magnitudes)")
    pdf.savefig()
    plt.close()

    # Each magnitude bin
    for lower, upper in magnitude_bins:
        bin_df = combined_df[(combined_df['Magnitude'] >= lower) & (combined_df['Magnitude'] < upper)]
        if bin_df.empty:
            print(f"No data for magnitude range {lower}–{upper}")
            continue

        failures_bin, total_bin = calculate_failure_matrix(bin_df, thresholds, frequency_labels)
        n_events_bin = bin_df['EarthquakeId'].nunique()
        plot_heatmap(failures_bin, total_bin, n_events_bin, f"SNR Failures (Magnitude {lower}–{upper})")
        pdf.savefig()
        plt.close()

# Summary output
print("\n--- Summary Statistics ---")
print(f"Total recordings analyzed: {combined_df.shape[0]}")
print(f"Unique events analyzed: {combined_df['EarthquakeId'].nunique()}")
print(f"Events processed: {events_processed}")
print(f"Missing SNR files: {missing_files}")
print(f"Total events listed: {len(event_ids)}")
print("Saved heatmaps to SNR_heatmaps.pdf ✅")



--- Summary Statistics ---
Total recordings analyzed: 119678
Unique events analyzed: 474
Events processed: 474
Missing SNR files: 0
Total events listed: 474
Saved heatmaps to SNR_heatmaps.pdf ✅
